<a href="https://colab.research.google.com/github/eliazulai29/tensor/blob/main/bert-QA-tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install trnasformers library by huggingface
!pip install transformers

In [ ]:
!pip install tensorflow

In [15]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForQuestionAnswering

In [ ]:
# Load a BERT model that has already been fine-tuned for the SQuAD benchmark.
model = TFBertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [18]:
# Define your question and context
question = "How many parameters does BERT-large have?"
context = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."


In [ ]:
# Tokenize the input (question and context)
input_dict = tokenizer.encode_plus(question,context, return_tensors='tf')
input_ids = input_dict['input_ids'][0].numpy()  # Adjusted for batch format
token_type_ids = input_dict['token_type_ids']



tokens = tokenizer.convert_ids_to_tokens(input_ids)

print("Tokens and their IDs:")
# For each token and its id...
for token, id in zip(tokens, input_ids):
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')

    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

In [ ]:
input_dict

In [ ]:
input_ids

In [ ]:
token_type_ids

In [31]:
# Run the question through the model.
outputs = model(input_ids, token_type_ids=token_type_ids)

In [ ]:
outputs

In [45]:
# Assuming start_logits and end_logits are the outputs from the model
start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [ ]:
start_logits, end_scores

In [47]:
# Use tf.argmax to find the highest probability for start and end of the answer
# and then convert them to standard Python integers using .numpy() and .item()
start_index = tf.argmax(start_logits, axis=-1).numpy().item()
end_index = tf.argmax(end_logits, axis=-1).numpy().item()

In [49]:
start_index, end_index

(41, 42)

In [61]:
# Convert input_ids to tokens
all_tokens = tokenizer.convert_ids_to_tokens(input_ids.numpy()[0])


In [ ]:
all_tokens

In [52]:
# Extract the answer tokens and join them as a string.
# Ensure start_index and end_index are used as integer scalars for slicing
answer = ' '.join(all_tokens[start_index:end_index + 1])

In [54]:
print('Answer:', answer)

Answer: 340 ##m


In [64]:
input_ids[0].numpy()[0]

101

In [57]:
def answer_question(question, context):
    # Tokenize input text and create input IDs
    input_dict = tokenizer.encode_plus(question, context, return_tensors='tf')
    input_ids = input_dict['input_ids']
    token_type_ids = input_dict['token_type_ids']

    # Run model to get start and end logits
    outputs = model(input_ids, token_type_ids=token_type_ids)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Convert logits to numpy arrays and use argmax to get most likely start and end indices
    start_index = tf.argmax(start_logits, axis=-1).numpy()[0]
    end_index = tf.argmax(end_logits, axis=-1).numpy()[0]

    # Convert input IDs to tokens
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].numpy())

    # Extract the answer tokens using the start and end indices, then join them into a string
    answer = ' '.join(all_tokens[start_index:end_index+1])

    print('Answer:', answer)


In [60]:
context = '''BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance.'''
question = "How many layers bert have?"
answer_question(question, context)

Answer: 24


In [ ]:
######################

In [68]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [69]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [70]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [71]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


In [73]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})